In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.request import Request, urlopen
from urllib.parse import urlparse
from urllib.error import HTTPError, URLError
import requests
import logging
from socket import timeout
import re

In [2]:
politifact_fake = pd.read_csv('fakenewsnet/politifact_fake.csv')

In [3]:
politifact_fake

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...
...,...,...,...,...
427,politifact14944,http://thehill.com/homenews/senate/369928-who-...,Who is affected by the government shutdown?,954602090462146560\t954602093171609600\t954650...
428,politifact14071,https://web.archive.org/web/20170322070001/htt...,Lindsey Graham Threatens To Convert To Democra...,740351669834244096\t740391312277573632\t740474...
429,politifact13577,http://archive.is/DQqCx,ELECTORAL COLLEGE ELECTOR COMMITS SUICIDE TO A...,810939651900907520\t814738996706164737\t814917...
430,politifact13560,politicono.com/palin-calls-boycott-mall-americ...,Sarah Palin Calls To Boycott Mall Of America B...,805216591424212993\t805222830023540736\t805223...


In [7]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def process_url(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    try:
        html = urlopen(req, timeout = 10)
        html = html.read()
        return text_from_html(html)
    except timeout:
        print("connection timedout")
    except URLError as e:
        print(e)
    except HTTPError as err:
        if err.code == 410 or 404:
            print("permanently deleted or removed, url should be removed")
        else:
            raise

In [8]:
# politifact_fake['text'] = ''
for index, url in enumerate(tqdm(politifact_fake['news_url'])):
    if politifact_fake['text'].iloc[index] == '':
        if type(url) != float:
            if urlparse(url).scheme:
                if politifact_fake['text'].iloc[index] == '':
                    politifact_fake['text'].iloc[index] = process_url(url)

  0%|          | 0/432 [00:00<?, ?it/s]

HTTP Error 410: Gone
HTTP Error 404: Not Found
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 410: Gone
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1124)>
HTTP Error 406: Not Acceptable
<urlopen error [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1124)>
HTTP Error 404: Not Found
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 404: Not Found
<urlopen error [Errno 61] Connection refused>
HTTP Error 410: Gone
<urlopen error [Errno 8] nodename nor servname provided, or not known>
HTTP Error 406: Not Acceptable
<urlopen error [Errno 8] nodename nor servname provided, or not known>
<urlopen error [Err

In [9]:
filtered_p_fake = politifact_fake[politifact_fake['text'] != '' ]
filtered_p_fake = filtered_p_fake[filtered_p_fake['text'].notnull()]
filtered_p_fake.to_csv('data/politifact_fake_with_scraped_text.csv')

In [10]:
# filtered_p_fake

In [11]:
from gensim.corpora import Dictionary

MIN_DF = 1 # minium document frequency
MAX_DF = 0.6 # maximum document frequency

/Users/vangelistrikoupis/miniforge3/envs/tf24/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [12]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')


In [13]:
tokenized_texts = []
for index, text in tqdm(enumerate(filtered_p_fake['text'])):
    tokenized_texts.append(tokenizer.tokenize(text.lower()))
        
# tokenized_text = tokenizer.tokenize(sample_text.lower())
# print(tokenized_text)

0it [00:00, ?it/s]

In [14]:
dictionary = Dictionary(tokenized_texts) # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF, 
                           no_above=MAX_DF)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

In [41]:
from gensim.models.wrappers import LdaMallet

PATH_TO_MALLET = 'Mallet/bin/mallet'
N_TOPICS = 2
N_ITERATIONS = 1000

lda = LdaMallet(PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                iterations=N_ITERATIONS)

ImportError: cannot import name 'Vocab' from 'gensim.models.word2vec' (/Users/vangelistrikoupis/miniforge3/envs/tf24/lib/python3.8/site-packages/gensim/models/word2vec.py)